In [1]:
%matplotlib inline
import matplotlib.pyplot as plt, mpld3, glob, os
import pandas as pd, seaborn as sn, numpy as np
sn.set_context('talk')

# Krypsiv (part 3)

A [previous notebook](http://nbviewer.jupyter.org/github/JamesSample/krypsiv/blob/master/krypsiv.ipynb) explored various plotting options for the krypsiv data, and Therese has since chosen the plots she'd like to use in the final report (see e-mail received 02/11/2016 at 09:54). In a second notebook (`krypsiv_2.ipynb`), I've modified my code to produce tidier versions of the required plots and saved them all as PNGs.

Rather than linking to the visualisations in the second notebook, I think it's probably better to view the PNG plots themselves, as the appearance online is sometimes slightly different to the finished image files. Ultimately, it's the PNGs that will be used in the report, so these versions need the most careful checking.

My aims with this notebook are:

 1. Provide an overview of the plots generated so far and highlight a few things that need checking, and <br><br>
 
 2. To take a look at some of the other data processing tasks (e.g. catchment area calculations, periods when the water level sensors are above the water etc.). 
 
**NB:** We're still waiting for some data from the lab, so some of these plots will change in the future.

**NB2:** I still haven't produced plots showing the variance of the replicate pH, redox and conductivity measurements (point 6 in Therese's e-mail). **Come back to this**.
 
## 1. PNG plots

I don't have permission to upload files to the project folder on the network, so I've created a temporary folder for the plots here:

K:\Prosjekter\JES\TFM_Krypsiv\Plots

This folder currently contains 8 PNG files and a subfolder called `chem_ts_by_site`, which contains a further 19 PNGs.

**Things to check:**

 1. For all files, please check the axis labels, units (if appropriate) and my Norwegian (!). <br><br>
 
 2. For *algal_proportions.png*, all the proportions add to 100%, except for station HIL during 2014. Is this correct? <br><br>
 
 3. For *substrate_proportions.png*, I've adjusted the "Organisk" class to make the totals sum to 100% (as suggested by Therese in her e-mail). For each plot, asterisks (\*) on the years on the x-axis highlight where adjustments have been made. The following results still look a bit strange and need further checking:
 
  * VAL in 2014 has 200% boulders *and* 100% organic. The overall total is currently 380% 
  * Even removing the organic column, the total for VAL in 2016 is 110%, so one of the other columns also needs reducing 
  * SØR in 2016 currently sums to 157%, but with only 15% organic. Some of the other classes will therefore need reducing too
  
## 2. Estimate discharge for MID and FYG

Based on the preliminary analysis in section 3 of the [previous notebook](http://nbviewer.jupyter.org/github/JamesSample/krypsiv/blob/master/krypsiv.ipynb), it looks reasonable to estimate discharge at stations MID and FYG by area-scaling the flow data from the NVE site at Kjølemo.

I've used the map in *Fig. 7* of the 2015 krypsiv report to identify the approximate locations of FIG and MID on the [NEVINA website](http://nevina.nve.no/), which gives the following catchment areas:

| Site | Area ($km^2$) |
|:----:|:-------------:|
|  MID |        1787.3 |
|  FYG |        1534.1 |

According to NVE, the catchment area for Kjølemo is $1757.7 \; km^2$. Based on this, we can estimate flow at FYG and MID using the code below (the pan and zoom tools can be used to explore the plot, as before).

The NEVINA website also produces PDFs of catchment properties, which may conatin useful information. For MID and FYG, these can be found here:

K:\Prosjekter\JES\TFM_Krypsiv\Catchment_Props

In [2]:
# Catchment areas (km2)
kjo_area = 1757.7
mid_area = 1787.3
fyg_area = 1534.1

# Read Kjølemo flow data from NVE
kjo_xls = r'C:\Data\James_Work\Staff\Therese_FM\Bulbous_Rushes\Data\Tidied\nve_kjolemo_22-4-0.xlsx'

df = pd.read_excel(kjo_xls, sheetname='Sheet1',
                   index_col=0)

df = df.resample('D').mean()

df.dropna(how='any', inplace=True)

# Area scaling
df['MID'] = df['kjo_q_m3ps'] * mid_area / kjo_area
df['FYG'] = df['kjo_q_m3ps'] * fyg_area / kjo_area

del df['kjo_q_m3ps']

# Plot
df.plot(figsize=(12,6))
plt.xlabel('')
plt.ylabel('Discharge (m3/s)')
plt.title('Estimated daily discharge')
mpld3.display()

## 3. Periods when the pressure sensors are dry

In an e-mail received 02/11/2016 at 12:50, Therese asked for a way of summarising periods when the water level pressure sensors are out of the water. The easiest way to do this is probably to look at the proportion of zero and negative values in each series, to get an idea of which stations dry out most often.

Kate has uploaded all the logger data to a folder here:

K:\Avdeling\318 Ferskvannsøkologi\Prosjekter\2016 JBovervåk16-18 (TFM)\04 Vannføring\HOBO\Calibrated_hobologgers

I've made a local copy of this data and then editied it so that I can read it in easily below. The tidied version is here:

C:\Data\James_Work\Staff\Therese_FM\Bulbous_Rushes\Data\Tidied\stage

In the code below, the raw data for each series is read in first, then any "no data" values are removed and the percentage of each record where values are less than or equal to zero is calculated and displayed.  

In [3]:
# Stage folder
data_fold = r'C:\Data\James_Work\Staff\Therese_FM\Bulbous_Rushes\Data\Tidied\stage'

# Site names
stns = ['APA', 'ASE', 'BJO', 'BRO', 'DRA', 'FYG', 
        'HER', 'HIL', 'KIL', 'LOG', 'MID', 'OSE', 
        'SME', 'SOR', 'STR', 'SVE', 'VAL', 'VEN']

# Container for output
dry_dict = {'Pct_Dry':[]}

# loop over sites
for stn in stns:
    # Some sites only have data for 2015
    if stn in ('VEN', 'SME'):
        # Read just 2015 data
        path_2015 = os.path.join(data_fold, '%s_2015.csv' % stn)
        df = pd.read_csv(path_2015, index_col=0, 
                         parse_dates=True,
                         dayfirst=True,
                         delimiter=';')
    
    else: # Have files for 2015 and 2016
        # 2015
        path_2015 = os.path.join(data_fold, '%s_2015.csv' % stn)
        df_2015 = pd.read_csv(path_2015, index_col=0, 
                              parse_dates=True,
                              dayfirst=True,
                              delimiter=';')      

        # 2016
        path_2016 = os.path.join(data_fold, '%s_2016.csv' % stn)
        df_2016 = pd.read_csv(path_2016, index_col=0, 
                              parse_dates=True,
                              dayfirst=True,
                              delimiter=';')  
      
        # Concatenate
        df = pd.concat([df_2015, df_2016], axis=0)
       
    # Drop any NoData values
    df.dropna(how='any', inplace=True)

    # Add % of values <= 0 to dict
    n_dry = (df <= 0).sum().values[0]
    prop = 100.*n_dry / len(df)
    dry_dict['Pct_Dry'].append(prop)

# Build df
dry_df = pd.DataFrame(data=dry_dict, index=stns)
dry_df.sort_values(by='Pct_Dry', ascending=False, inplace=True)

print 'Percentages of zero and negative values in each water level series:'
dry_df

Percentages of zero and negative values in each water level series:


,Pct_Dry
ASE,9.665386
LOG,5.546796
SME,1.159107
FYG,1.070513
MID,1.067853
SVE,0.863317
BRO,0.636090
VEN,0.491685
BJO,0.324657
OSE,0.251214
